In [110]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [111]:
# Load the original dataset
df = pd.read_csv('../data/customer.csv')
print(df.shape)
print(df.columns)

(72458, 15)
Index(['Unnamed: 0', 'custid', 'sex', 'is_employed', 'income',
       'marital_status', 'health_ins', 'housing_type', 'num_vehicles', 'age',
       'state_of_res', 'code_column', 'gas_usage', 'rooms', 'recent_move_b'],
      dtype='object')


In [112]:
# Check the number of missing values in 'is_employed'
df['is_employed'].value_counts(dropna=False)

is_employed
True     44630
NaN      25515
False     2313
Name: count, dtype: int64

In [113]:
# make is_emplyed categorical with 0 and 1, and replace NaN with 2
# 0 is for unemployed, 1 is for employed, 2 is for people who are not in workforce
df['is_employed'] = df['is_employed'].map({True: 1, False: 0}).fillna(0)
df['is_employed'].value_counts()

is_employed
1.0    44630
0.0    27828
Name: count, dtype: int64

In [114]:
df.groupby('state_of_res')['code_column'].nunique()

state_of_res
Alabama                 1
Alaska                  1
Arizona                 1
Arkansas                1
California              1
Colorado                1
Connecticut             1
Delaware                1
District of Columbia    1
Florida                 1
Georgia                 1
Hawaii                  1
Idaho                   1
Illinois                1
Indiana                 1
Iowa                    1
Kansas                  1
Kentucky                1
Louisiana               1
Maine                   1
Maryland                1
Massachusetts           1
Michigan                1
Minnesota               1
Mississippi             1
Missouri                1
Montana                 1
Nebraska                1
Nevada                  1
New Hampshire           1
New Jersey              1
New Mexico              1
New York                1
North Carolina          1
North Dakota            1
Ohio                    1
Oklahoma                1
Oregon                  1

In [115]:
df['code_column'].value_counts()

code_column
8962    8870
6026    5938
4979    4921
4431    4375
2997    2968
2925    2896
2614    2587
2357    2329
2269    2246
2198    2177
2059    2035
1942    1922
1681    1663
1642    1635
1534    1510
1501    1489
1467    1452
1397    1388
1337    1324
671     1324
1305    1294
1291    1274
1226    1214
1177    1167
1104    1085
1047    1037
952      942
935      923
901      889
832      827
706      702
665      657
653      646
647      638
422      416
407      402
404      399
354      354
170      337
333      328
325      319
307      305
220      218
218      216
204      198
188      186
162      160
146      146
131      130
Name: count, dtype: int64

In [116]:
# Since each state has 1 code, we can drop the 'code_column' feature. We can also drop 'Unnamed: 0' feature
df.drop(['Unnamed: 0','custid','code_column','recent_move_b'], axis=1, inplace=True)
print(df.shape)
print(df.columns)

(72458, 11)
Index(['sex', 'is_employed', 'income', 'marital_status', 'health_ins',
       'housing_type', 'num_vehicles', 'age', 'state_of_res', 'gas_usage',
       'rooms'],
      dtype='object')


In [117]:
df.isnull().sum()

sex                  0
is_employed          0
income               0
marital_status       0
health_ins           0
housing_type      1686
num_vehicles      1686
age                  0
state_of_res         0
gas_usage         1686
rooms                0
dtype: int64

In [118]:
num = df[df.isnull().any(axis=1)].shape[0]
print(print(f'{num} rows have missing values. \nApprox. {num/df.shape[0]*100:.2f}% of the orignal dataset.'))

1686 rows have missing values. 
Approx. 2.33% of the orignal dataset.
None


In [119]:
# Missing values are all in the same rows. We can drop them
df.dropna(inplace=True)
df.shape

(70772, 11)

In [120]:
# Filter out the outliers

# columns_with_outliers = ['age', 'income', 'gas_usage']

# for col in columns_with_outliers:
#     q1 = df[col].quantile(0.25)
#     q3 = df[col].quantile(0.75)
#     iqr = q3 - q1
#     lower_bound = q1 - 1.5 * iqr
#     upper_bound = q3 + 1.5 * iqr
#     print(f'{col}: {df[(df[col] < lower_bound) | (df[col] > upper_bound)].shape[0]} outliers')
#     df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]


# df.shape

In [121]:
le = LabelEncoder()
cat_columns = ['sex', 'marital_status', 'health_ins', 'housing_type', 'state_of_res']

for col in cat_columns:
    df[col] = le.fit_transform(df[col])

df.head(10)

,sex,is_employed,income,marital_status,health_ins,housing_type,num_vehicles,age,state_of_res,gas_usage,rooms
0,1,1.0,22000.0,2,1,0,0.0,24,0,210.0,3
1,0,0.0,23200.0,0,1,3,0.0,82,0,3.0,6
2,0,1.0,21000.0,2,1,1,2.0,31,0,40.0,3
3,0,0.0,37770.0,3,1,0,1.0,93,0,120.0,2
4,1,1.0,39000.0,0,1,3,2.0,67,0,3.0,2
5,1,0.0,11100.0,1,1,0,2.0,76,0,200.0,6
6,0,1.0,25800.0,1,0,3,2.0,26,0,3.0,3
7,0,0.0,34600.0,1,1,0,2.0,73,0,50.0,5
8,0,1.0,25000.0,2,1,0,5.0,27,0,3.0,4
9,1,1.0,31200.0,1,1,1,3.0,54,0,20.0,6


In [122]:
df[df.age < 21].age.value_counts()

age
0    76
Name: count, dtype: int64

In [123]:
df[df.age < 21]

,sex,is_employed,income,marital_status,health_ins,housing_type,num_vehicles,age,state_of_res,gas_usage,rooms
594,1,1.0,50000.0,2,0,3,1.0,0,0,3.0,3
1260,1,0.0,0.0,1,1,3,0.0,0,2,3.0,4
1658,0,1.0,24700.0,2,1,3,3.0,0,2,3.0,5
2340,0,1.0,2400.0,0,1,3,0.0,0,2,3.0,4
2859,0,0.0,9700.0,1,1,0,3.0,0,3,3.0,2
...,...,...,...,...,...,...,...,...,...,...,...
67967,0,0.0,5000.0,3,1,1,0.0,0,46,3.0,2
68681,0,1.0,80000.0,1,1,1,2.0,0,46,90.0,3
69200,1,0.0,0.0,2,1,3,2.0,0,47,3.0,6
70015,1,1.0,75000.0,0,1,0,2.0,0,47,3.0,4


In [124]:
# delete rows where age == 0 and health_ins == 1
df = df[~((df.age == 0) & (df.health_ins == 1))]
df[df.age < 21]

,sex,is_employed,income,marital_status,health_ins,housing_type,num_vehicles,age,state_of_res,gas_usage,rooms
594,1,1.0,50000.0,2,0,3,1.0,0,0,3.0,3
42750,1,1.0,20000.0,1,0,3,1.0,0,31,3.0,4


In [125]:
scaler = MinMaxScaler()
df[['age', 'gas_usage']] = scaler.fit_transform(df[['age', 'gas_usage']])

df.head(10)

,sex,is_employed,income,marital_status,health_ins,housing_type,num_vehicles,age,state_of_res,gas_usage,rooms
0,1,1.0,22000.0,2,1,0,0.0,0.200000,0,0.367311,3
1,0,0.0,23200.0,0,1,3,0.0,0.683333,0,0.003515,6
2,0,1.0,21000.0,2,1,1,2.0,0.258333,0,0.068541,3
3,0,0.0,37770.0,3,1,0,1.0,0.775000,0,0.209139,2
4,1,1.0,39000.0,0,1,3,2.0,0.558333,0,0.003515,2
5,1,0.0,11100.0,1,1,0,2.0,0.633333,0,0.349736,6
6,0,1.0,25800.0,1,0,3,2.0,0.216667,0,0.003515,3
7,0,0.0,34600.0,1,1,0,2.0,0.608333,0,0.086116,5
8,0,1.0,25000.0,2,1,0,5.0,0.225000,0,0.003515,4
9,1,1.0,31200.0,1,1,1,3.0,0.450000,0,0.033392,6


In [126]:
df['income'] = (df['income']-df['income'].mean())/df['income'].std()

df.income.describe()

count    7.069800e+04
mean    -5.387011e-17
std      1.000000e+00
min     -8.445397e-01
25%     -5.342559e-01
50%     -2.580693e-01
75%      1.602171e-01
max      2.070313e+01
Name: income, dtype: float64

In [127]:
# save the cleaned data to a new csv file
df.to_csv('../data/customer_cleaned.csv', index=False)